# 데이터 합치기 & 간단한 형식 맞추기

- [간단 전처리 함수](#간단-전처리)
- [완결 웹툰 concat](#완결-웹툰)
- [연재중 웹툰 concat](#연재중-웹툰)
- [데이터 합치기 & 내보내기](#완결,-연재중-웹툰-구분하는-column-추가-후,-합치기)

In [1]:
import pandas as pd
import numpy as np
import re

import os

In [2]:
# 파일 이름 가져오는 함수
def get_file_names(folder_path):
    files = os.listdir(folder_path)
    return files

folder_path = '../1.데이터 수집/collected_data/' # 특정 폴더의 경로 지정
file_names = get_file_names(folder_path) # 파일 이름 가져오기

# 결과 출력
fin_folder_lst = [] # 완결 웹툰 csv
con_folder_lst = [] # 연재중 웹툰 csv
for name in file_names:
    if name.split('.')[-1] == 'csv':
        if name.split('.')[0].split('_')[0] == 'fin':
            fin_folder_lst.append(name)
        else:
            con_folder_lst.append(name)

In [3]:
# fin_folder_lst[:-4] # 뒤에 네개는 info와 score
# con_folder_lst[:-6] # 뒤에 여섯개는 info와 score

# 완결 웹툰 comment 데이터 합치기
fin_comment=pd.DataFrame()
for i in fin_folder_lst[:-4]:
    d = pd.read_csv(f'collected_data/{i}')
    fin_comment=pd.concat([fin_comment,d])
    
# 연재중 웹툰 comment 데이터 합치기
con_comment=pd.DataFrame()
for i in con_folder_lst[:-6]:
    d = pd.read_csv(f'collected_data/{i}')
    con_comment=pd.concat([con_comment,d])

# 간단 전처리

In [4]:
# title에 휴재,\n 있는 경우, 제외하기
def Name(text):
    result = re.sub('\n', '', string=text)
    result = re.sub('휴재', '', string=result)
    return result

In [5]:
# total 전체 화 숫자로 뽑기
def Extract_num(text):
    text = str(text) # str로 변환해줘야 type error가 안뜸
    pattern = r'\d+'  # 1개 이상의 연속된 숫자에 매치되는 패턴
    numbers = re.findall(pattern, text)
    return numbers[0]

In [6]:
# 장르 #장르 형식으로 만들기
def Genre(text):
    tt = ''
    p = re.compile('(?<=#)[가-힣.]+')
    result = p.findall(text)
    for i in result:
        i = '#'+i
        tt+=i
    return tt

In [7]:
def Descript(text):
    result = re.sub('\n', '', string=text)
    return result

In [8]:
# origin의 'X'를 'No_original'로 바꾸기
def Original(text):
    if text == 'X':
        text = 'No_original'
    return text

In [10]:
# author, artist, origin 작가들 #작가 형식으로 저장
def Writer(text):
    tt = ''
    result = text.split(', ')
    for i in result:
        i = '#'+i
        tt+=i
    return tt

# 완결 웹툰

In [11]:
fin_info = pd.read_csv('collected_data/fin_webtoon_info.csv').iloc[:,1:]
fin_score = pd.read_csv('collected_data/fin_score_data.csv')

# 시간에 따라 바뀌기 때문에 정보가 안들어 간 경우가 있어서 마지막에 한번 더 추출한 정보
fin_info_plus = pd.read_csv('collected_data/fin_webtoon_info_plus.csv').iloc[:,1:]
fin_score_plus = pd.read_csv('collected_data/fin_score_data_plus.csv')

In [12]:
# info 전처리
fin_info['title'] = fin_info['title'].apply(Name)
fin_info['total'] = fin_info['total'].apply(Extract_num)

fin_info_plus['title'] = fin_info_plus['title'].apply(Name)
fin_info_plus['total'] = fin_info_plus['total'].apply(Extract_num)

# score 전처리
fin_score['title'] = fin_score['title'].apply(Name)
fin_score_plus['title'] = fin_score_plus['title'].apply(Name)

# comment 전처리
fin_comment['title'] = fin_comment['title'].apply(Name)

# 댓글이 없는 경우를 위해 Null값 치환
fin_comment['best_comment'] = fin_comment['best_comment'].fillna("\n")
fin_comment['comment'] = fin_comment['comment'].fillna("\n")

In [13]:
fin_info=pd.concat([fin_info,fin_info_plus])
fin_info.drop_duplicates(subset = ['title','descript','genre'],inplace=True) # 중복 row 제거
fin_info = fin_info.reset_index().iloc[:,1:]

fin_score=pd.concat([fin_score,fin_score_plus])
fin_score.drop_duplicates(subset = ['title','episode'],inplace=True) # 중복 row 제거
fin_score = fin_score.reset_index().iloc[:,1:]

fin_comment.drop_duplicates(inplace=True) # 중복 row 제거
fin_comment = fin_comment.reset_index().iloc[:,1:]

In [14]:
# 하나의 데이터로 합치기
final = pd.merge(fin_comment,fin_score, on=['title','episode'], how='left')
final = pd.merge(final,fin_info,on='title', how='left')

In [15]:
# 위에서 정의한 함수 사용
final['genre'] = final['genre'].apply(Genre)
final['descript'] = final['descript'].apply(Descript)
final['origin'] = final['origin'].apply(Original)
final['author'] = final['author'].apply(Writer)
final['artist'] = final['artist'].apply(Writer)
final['origin'] = final['origin'].apply(Writer)

# 연재중 웹툰

In [16]:
con_info = pd.read_csv('collected_data/continue_webtoon_info.csv').iloc[:,1:]
con_info_plus = pd.read_csv('collected_data/continue_webtoon_info_plus.csv').iloc[:,1:]
con_info_plus2 = pd.read_csv('collected_data/continue_webtoon_info_plus2.csv').iloc[:,1:]
con_score = pd.read_csv('collected_data/continue_score_data.csv')
con_score_plus = pd.read_csv('collected_data/continue_score_data_plus.csv')
con_score_plus2 = pd.read_csv('collected_data/continue_score_data_plus2.csv')

In [17]:
# info 전처리
con_info['title'] = con_info['title'].apply(Name)
con_info['total'] = con_info['total'].apply(Extract_num)

con_info_plus['title'] = con_info_plus['title'].apply(Name)
con_info_plus['total'] = con_info_plus['total'].apply(Extract_num)

con_info_plus2['title'] = con_info_plus2['title'].apply(Name)
con_info_plus2['total'] = con_info_plus2['total'].apply(Extract_num)

# score 전처리
con_score['title'] = fin_score['title'].apply(Name)
con_score_plus['title'] = con_score_plus['title'].apply(Name)
con_score_plus2['title'] = con_score_plus2['title'].apply(Name)

# comment 전처리
con_comment['title'] = con_comment['title'].apply(Name)

# 댓글이 없는 경우를 위해 Null값 치환
con_comment['best_comment'] = con_comment['best_comment'].fillna("\n")
con_comment['comment'] = con_comment['comment'].fillna("\n")

In [18]:
con_info=pd.concat([con_info,con_info_plus,con_info_plus2])
con_info.drop_duplicates(subset = ['title','day','descript','genre'],inplace=True) # 중복 row 제거
con_info = con_info.reset_index().iloc[:,1:]

con_score=pd.concat([con_score,con_score_plus,con_score_plus2])
con_score.drop_duplicates(subset = ['title','day','episode'],inplace=True) # 중복 row 제거
con_score = con_score.reset_index().iloc[:,1:]

con_comment.drop_duplicates(inplace=True) # 중복 row 제거
con_comment = con_comment.reset_index().iloc[:,1:]

In [19]:
# 하나의 데이터로 합치기
serialized = pd.merge(con_comment,con_score, on=['title','episode'], how='left')
serialized = pd.merge(serialized,con_info,on=['title','day'], how='left')

In [20]:
# 위에서 정의한 함수 사용
serialized['genre'] = serialized['genre'].apply(Genre)
serialized['descript'] = serialized['descript'].apply(Descript)
serialized['origin'] = serialized['origin'].apply(Original)
serialized['author'] = serialized['author'].apply(Writer)
serialized['artist'] = serialized['artist'].apply(Writer)
serialized['origin'] = serialized['origin'].apply(Writer)

# 완결, 연재중 웹툰 구분하는 column 추가 후, 합치기

In [21]:
# 각 에피소드 평점과 해당 웹툰의 전체 평균과 비교하여 sentiment column 추가
final['sentiment'] = final['score']>=final['star']
final['sentiment'] = final['sentiment'].astype(int)
final['day'] = '완결'

serialized['sentiment'] = serialized['score']>=serialized['star']
serialized['sentiment'] = serialized['sentiment'].astype(int)

final['web'] = '완결'
serialized['web'] = '연재중'

final = final[['web', 'title', 'day', 'episode', 'genre', 'descript', 
               'author', 'artist', 'origin', 'total', 'star', 'date', 
               'best_comment', 'comment', 'comment_num', 'score','sentiment']]
serialized = serialized[['web', 'title', 'day', 'episode', 'genre', 'descript', 
                         'author', 'artist', 'origin', 'total', 'star', 'date', 
                         'best_comment', 'comment', 'comment_num', 'score','sentiment']]

In [22]:
webtoon_data = pd.concat([final,serialized],axis = 0)

In [28]:
webtoon_data.to_csv('../data/webtoon_data.csv', index=False)

In [27]:
webtoon_data.head(3)

,web,title,day,episode,genre,descript,author,artist,origin,total,star,date,best_comment,comment,comment_num,score,sentiment
0,완결,연애혁명,완결,preseason 1. 모든 첫 만남은 대부분 우연이다,#드라마#다정남#혐관로맨스#하이틴#학원로맨스#완결무료#완결드라마,평범하면서 금사빠인 고등학생 순정남 공주영은 까칠하고 차가운 여학생 왕자림을 보고 ...,#232,#232,#No_original,442,9.86,13.09.04,왔구나!!!!\n베도의 전설 연애혁명이 드디어 혁명을 일으킨다독자들은 풍악을 울리고...,연혁이 끝이라고?..진짜?..나만 안믿겨?..\n어흑 ㅠㅠㅠ 내 10년\n유료 되기...,51695,9.89,1
1,완결,연애혁명,완결,preseason 2. 간혹가다 호의를 호감으로 착각할 때가 있다,#드라마#다정남#혐관로맨스#하이틴#학원로맨스#완결무료#완결드라마,평범하면서 금사빠인 고등학생 순정남 공주영은 까칠하고 차가운 여학생 왕자림을 보고 ...,#232,#232,#No_original,442,9.86,13.09.11,목요웹툰에 대작이 늘어나고있다\n공개해야하는정봌ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 진격의거인ㅋㅋ...,드라마랑 같이 보니 더 꿀잼!\n92년생?\n이때는 어땠을까요\n병훈아 왜 경우한테...,15379,9.91,1
2,완결,연애혁명,완결,preseason 3. 용기있는 자만이 미인을 차지한다?,#드라마#다정남#혐관로맨스#하이틴#학원로맨스#완결무료#완결드라마,평범하면서 금사빠인 고등학생 순정남 공주영은 까칠하고 차가운 여학생 왕자림을 보고 ...,#232,#232,#No_original,442,9.86,13.09.18,왠지 작가의 말 보고 별점주고 싶어졌어\n니가 뭘좋아할지몰라서 담임을준비햇어(수줍)...,지난 4개월을 흘려보냈네요\n30살때였음\n정주행하는데 이게 내가 학생때 나온 작품...,13075,9.91,1
